In [1]:
function hotstart_michelot(d::AbstractVector, b::Real = 1)
    y = d[1:10_000]
    p = (sum(y)-b)/10_000
    while true
        l = length(y)
        s = -b
        for i in 1:l
            x = popfirst!(y)
            if x>p
                push!(y, x)
                s += x
            end
        end
        p = s/length(y)
        if length(y) == l
            break
        end    
    end
    y = d[1:end]
    while true
        l = length(y)
        s = -b
        for i in 1:l
            x = popfirst!(y)
            if x>p
                push!(y, x)
                s += x
            end
        end
        p = s/length(y)
        if length(y) == l
            break
        end 
    end
    return max.(d.-p, 0)
end

hotstart_michelot (generic function with 2 methods)

In [2]:
function michelot(d::AbstractVector, b::Real = 1)
    p = (sum(d)-b)/length(d)
    y = d[1:end]
    while true
        l = length(y)
        s = -b
        for i in 1:l
            x = popfirst!(y)
            if x>p
                push!(y, x)
                s += x
            end
        end
        p = s/length(y)
        if length(y) == l
            break
        end 
    end
    return max.(d.-p, 0)
end

michelot (generic function with 2 methods)

In [22]:
using BangBang
function serial_filter(y::AbstractVector, b::Real = 1)
    #initialize active list
    v = [y[1]]
    #initialize pivot
    p = v[1]-b
    #initialize waiting list
    u = []
    sl = 0
        for i in 2:length(y)
        #remove inactive terms
        if y[i] > p
            #update pivot
            p = p + (y[i]-p)/(sl+=1)
            if p > y[i]-b
                push!(v, y[i])
            else
                #if pivot is too large, push previous list to waiting list
                append!!(u, v)
                v = [y[i]]
                p = y[i]-b
                sl = 1
            end
        end
    end
    #reuse some terms from waiting list
    for i in 1:length(u)
        if u[i] > p
                push!(v, u[i])
                p = p + (u[i]-p)/(sl+=1)
        end
    end   
    #output active list    
    return v
end

function checkL(y::AbstractVector, b::Real = 1)
    #initialize pivot, this step is used to improve numeric accuracy
    p = (sum(y)-b)/length(y)
    while true
        #record length
        l = length(y)
        #r is number of terms will be removed in this round
        r = 0
        #check all terms in current y
        s = -b
        for i in 1:l
            x = popfirst!(y)
            if x <= p
                r += 1
                #update pivot
                p = p + (p-x)/(l-r)
            else
                push!(y,x)
                s += x
            end
        end
        p = s/length(y)
        #check termination condition
        if l == length(y)
            return p
        end
    end
end

function serial_condat(d::AbstractVector, b::Real = 1)
    #filter
    y = serial_filter(d, b)
    #after filter, Condat's method scans and checks remaining rems
    p = checkL(y, b)
    #output projection result
    return max.(d.-p, 0)
    #return p
end

condat_s(data::AbstractVector, a::Real = 1) = serial_condat(data, a)

condat_s (generic function with 2 methods)

In [17]:
function hotstart_condat(d::AbstractVector, b::Real = 1)
    y = serial_filter(d[1:10_000], b)
    p = checkL(y, b)
    y = []
    s = -b
    for i in 1:length(d)
        if d[i]>p
            push!(y, d[i])
            s += d[i]
        end    
    end
    p = s/length(y)
    while true
        #record length
        l = length(y)
        #r is number of terms will be removed in this round
        r = 0
        #check all terms in current y
        s = -b
        for i in 1:l
            x = popfirst!(y)
            if x <= p
                r += 1
                #update pivot
                p = p + (p-x)/(l-r)
            else
                push!(y,x)
                s += x
            end
        end
        p = s/length(y)
        #check termination condition
        if l == length(y)
            return max.(d.-p, 0)
        end
    end
end    

hotstart_condat (generic function with 2 methods)

In [3]:
using BenchmarkTools, Random, Distributions
BenchmarkTools.DEFAULT_PARAMETERS.gcsample = true
BenchmarkTools.DEFAULT_PARAMETERS.samples = 100
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 20

20

In [4]:
Random.seed!(12345); @benchmark hotstart_michelot($(rand(1_00_000_000)), 1)

BenchmarkTools.Trial: 11 samples with 1 evaluation.
 Range (min … max):  1.558 s …   1.603 s  ┊ GC (min … max): 0.04% … 0.04%
 Time  (median):     1.570 s              ┊ GC (median):    0.04%
 Time  (mean ± σ):   1.575 s ± 16.970 ms  ┊ GC (mean ± σ):  0.07% ± 0.09%

  ▁  ▁ ▁ █       █             ▁                     ▁▁   ▁  
  █▁▁█▁█▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁██▁▁▁█ ▁
  1.56 s         Histogram: frequency by time         1.6 s <

 Memory estimate: 2.24 GiB, allocs estimate: 8.

In [5]:
Random.seed!(12345); @benchmark michelot($(rand(1_00_000_000)), 1)

BenchmarkTools.Trial: 5 samples with 1 evaluation.
 Range (min … max):  3.817 s …   4.004 s  ┊ GC (min … max): 0.02% … 0.02%
 Time  (median):     3.833 s              ┊ GC (median):    0.02%
 Time  (mean ± σ):   3.871 s ± 77.170 ms  ┊ GC (mean ± σ):  0.03% ± 0.01%

  █  ██           █                                       █  
  █▁▁██▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  3.82 s         Histogram: frequency by time           4 s <

 Memory estimate: 2.24 GiB, allocs estimate: 5.

In [13]:
Random.seed!(12345); @benchmark condat_s($(rand(1_00_000_000)), 1)

BenchmarkTools.Trial: 30 samples with 1 evaluation.
 Range (min … max):  347.939 ms … 357.469 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     349.664 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   350.245 ms ±   2.071 ms  ┊ GC (mean ± σ):  0.01% ± 0.04%

    ▃  █▃  ▃▃ ▃      ▃                                           
  ▇▇█▁▇██▁▇██▇█▇▁▇▁▇▇█▇▇▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇ ▁
  348 ms           Histogram: frequency by time          357 ms <

 Memory estimate: 764.94 MiB, allocs estimate: 20.

In [19]:
Random.seed!(12345); @benchmark hotstart_condat($(rand(1_00_000_000)), 1)

BenchmarkTools.Trial: 19 samples with 1 evaluation.
 Range (min … max):  758.944 ms … 775.829 ms  ┊ GC (min … max): 1.40% … 1.39%
 Time  (median):     761.515 ms               ┊ GC (median):    1.38%
 Time  (mean ± σ):   763.470 ms ±   5.254 ms  ┊ GC (mean ± σ):  1.39% ± 0.02%

  ▃    ▃▃  █                                                  ▃  
  █▇▁▇▁██▇▁█▁▁▁▁▇▁▇▁▁▇▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▇▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  759 ms           Histogram: frequency by time          776 ms <

 Memory estimate: 901.90 MiB, allocs estimate: 7987522.